### Initialize a tileset from a ND2 file

In [ ]:
from exm.stitching.tileset import Tileset

raw_data_path = '/mp/nas2/ruihan/20221017_alignment'
nd2_file = f"{raw_data_path}/20221017_alignment_slice1_final.nd2"
output_path = "/tmp/"

# Provide the voxel size so that conversions between pixel and µm coordinates can happen
ts=Tileset([0.1625*2, 0.1625*2, 0.4])

# This specifies what ND2 file to use as reference but does not load all the tiles. It uses lazy loading where the
# costly operation of loading all tiles in memory will happen only when needed. It initalizes offsets using the 
# metadata contained in the ND2 file (microscope's encoder data)

ts.init_from_nd2(nd2_file)

### Preview a Z-slice

This is a preview of a reconstructed slice, that is of a slice where each tile has been placed according to its initial offset. This is a fast operation that still does not load the tiles in memory but just accesses the XY slices needed for each tile

In [ ]:
from exm.utils import display_img as di 

zslice = ts.preview_nd2(150)
di(zslice.T)

### Load a downscaled version of the tiles in memory

This loads all the tiles into memory. If possible, try to downscale it as a full dataset can quickly fill up your memory.

In [ ]:
ts.load_all(downscale=[2,4,4])

### Scale intensities globally and converts to 8bpp

This finds an intensity range that is applicable accross the whole dataset and converts it to 8 bpp. This used to be useful for CysGAN segmentation.

In [ ]:
ts.scale_intensity()

In [ ]:
di(ts.show_slice(50).T)

### Produce an output volume

Produces a single numpy array containing the reconstructed 3D volume with each tile placed according to the offset

In [ ]:
vol = ts.produce_output_volume()

In [ ]:
vol.shape

In [ ]:
di(vol[50].T)

### Writing a H5/BDV file

This exports the tileset into a format that is readable by Fiji's BigStitcher plugin. The format is called H5 BigDataViewer. It consists of a H5 file containing all the tile images and a XML file containing the offsets information.

In [ ]:
ts.write_into_h5bdv(f"{output_path}/example_output.h5")

## Steps to refine this stitching in Fiji

### Headless mode
See https://github.com/mitmedialab/fishexseq/tree/main/headless_fiji

### Manual mode (through GUI)
1. Open Fiji
2. Do NOT use File->Open but rather go to Plugins->BigStitcher->BigStitcher
3. Select the XML file that was generated next to the H5 file. In our case `/tmp/example_output.xml`
4. Select all the tiles in the Stitching Explorer window
5. Right-click->Stitching wizard
6. Click OK with the default parameters
7. Say 'Yes' to "Switch to preview mode"
8. Click "Apply & Run Global Optimization"
9. Select the default strategy and click OK
10. Click "Save" (It will overwrite `/tmp/example_output.xml`)

In [ ]:
# At this stage `ts` still contains the old offsets

di(ts.show_slice(50).T)

In [ ]:
# We need to load the offset files produced by Fiji to overwrite the existing offsets
ts.update_offsets(f"{output_path}/example_output.xml")

In [ ]:
# We now get a reconstructed volume with less visible seams
di(ts.show_slice(50).T)